In [ ]:
from genet import design as ds

wt = 'GGTCACTCACCTGGAGTGAGCCCTGCTCCCCCCTGGCTCCTTCCCAGCCTGGGCATCCTTGAGTTCCAAGGCCTCATTCAGCTCTCGGAACATCTCGAAGCGCTCACGCCCACGGATCTGC'
ed = 'GGTCACTCACCTGGAGTGAGCCCTGCTCCCCCCTGGCTCCTTCCCAGCCTGGGCATCCTTGTTCCAAGGCCTCATTCAGCTCTCGGAACATCTCGAAGCGCTCACGCCCACGGATCTGCAG'
edit_type = 'del'
edit_len  = 2

pegrnas = ds.pegRNA(wt, ed, edit_type, edit_len)

In [ ]:
from genet import design as ds
# from genet.design.DesignUtils import dict_pam_disrup_rank, test_score_data

test_info, dict_test = ds.test_score_data('example_6')
ref_seq, frame, cds_start, cds_end = dict_test['ref_seq'], dict_test['frame'], dict_test['cds_start'], dict_test['cds_end'], 


df_out = ds.SynonymousPE(test_info, ref_seq=ref_seq, frame=frame, cds_start=cds_start, cds_end=cds_end)
df_out.output


In [ ]:
pegrnas.show_output()

In [ ]:
import random, sys, time, os

class BarcodeGenerator:
    '''Barcode sequence generator
    Using Primer3 and random module

    Design non-redundant barcode sequences using heterodimer melting temperature and homopolymer. 

    Written by Jin-Wu Nam Feb. 19. 2016
    '''

    def __init__(self, 
                barcode_len:int,
                num_of_barcode:int      = 100000,
                max_homopolymer_len:int = 4,
                max_hetero_dimer_tm:int = 30, # How to set Tm?
                diffn:int               = 2,  # maximum diff. ID
                line_n:int              = 1,
                cand_n:int              = 0,
                keep_barcodes_file:str      = None,
                ):
        
        import primer3
        print('Barcode generator by Jin-wu Nam')

        self.list_barcode        = []
        self.list_nt             = ['A', 'C', 'G', 'T']

        self.barcode_len         = barcode_len
        self.num_of_barcode      = num_of_barcode
        self.max_homopolymer_len = max_homopolymer_len
        self.max_hetero_dimer_tm = max_hetero_dimer_tm
        
        self.diffn               = diffn
        self.line_n              = line_n
        self.cand_n              = cand_n

        if keep_barcodes_file:
            if os.path.isfile(keep_barcodes_file):
                inputF = open(keep_barcodes_file, 'r')
                list_barcode = inputF.readlines()
                inputF.close()

            else:
                inputF = open(keep_barcodes_file, 'w')
                inputF.close()

    def run(self, ): # 기존 generate_barcode() 새로 추가할 것
        allL = self.generate_barcode()
        while not(self.check_homopolymer(allL)): 
            allL = self.generate_barcode()
        
        return ''.join(allL)


    def generate_barcode(self, barcode_len):
        list_nt = ['A','C','G', 'T']
        seq = ''

        for i in range(barcode_len):
            seq += list_nt[random.randint(0,3)]
        
        return seq

    def check_homopolymer(self, cand, max_homopolymer_len:int=4):
        homosize = 1
        prev='N'; flag=True

        for i in range(len(cand)):
            curr = cand[i]
            if prev==curr: 
                homosize+=1
                if homosize>max_homopolymer_len: homosize=1; flag=False; break
            #else: homosize=1
            prev=curr
        return flag

    def check_tm_and_align_num(self, cand):
        flag=True
        #print '.....................'
        for keep in KEEPLIST:
            res = primer3.calcHeterodimer(cand, keep)
            if res.tm > MaxHeteroDimerTm: flag=False; break
            #if res.align_end_1 > MaxMatchNum: flag=False; break
            #print res.tm, res.align_end_1
        return flag


    def check_identity(self, cand):
        flag=True; diff=0
        for keep in KEEPLIST:
            for i in range(len(keep)): 
                if cand[i]!=keep[i]: diff+=1
                if diff>DIFFN: break
            if diff<=DIFFN: flag=False; break
            else: diff=0
        return flag
        